In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
import io

In [2]:
books_df = pl.scan_ndjson('data/goodreads_books.json')

In [3]:
books_df.collect()

isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
str,str,list[str],str,str,list[struct[2]],str,str,str,str,list[str],str,str,str,list[struct[2]],str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""0312853122""","""1""",[],"""US""","""""","[{""3"",""to-read""}, {""1"",""p""}, … {""1"",""biography""}]","""""","""false""","""4.00""","""""",[],"""""","""Paperback""","""https://www.go…","[{""604031"",""""}]","""St. Martin's P…","""256""","""1""","""9780312853129""","""9""","""""","""1984""","""https://www.go…","""https://images…","""5333265""","""3""","""5400751""","""W.C. Fields: A…","""W.C. Fields: A…"
"""0743509986""","""6""",[],"""US""","""""","[{""2634"",""to-read""}, {""160"",""fiction""}, … {""2"",""general""}]","""""","""false""","""3.23""","""B000FC0PBC""","[""8709549"", ""17074050"", … ""307575""]","""Anita Diamant'…","""Audio CD""","""https://www.go…","[{""626222"",""""}]","""Simon & Schust…","""""","""1""","""9780743509985""","""10""","""Abridged""","""2001""","""https://www.go…","""https://s.gr-a…","""1333909""","""10""","""1323437""","""Good Harbor""","""Good Harbor"""
"""""","""7""","[""189911""]","""US""","""eng""","[{""58"",""to-read""}, {""15"",""fantasy""}, … {""1"",""read-in-my-20s""}]","""B00071IKUY""","""false""","""4.03""","""""","[""19997"", ""828466"", … ""1597281""]","""Omnibus book c…","""Hardcover""","""https://www.go…","[{""10333"",""""}]","""Nelson Doubled…","""600""","""""","""""","""""","""Book Club Edit…","""1987""","""https://www.go…","""https://images…","""7327624""","""140""","""8948723""","""The Unschooled…","""The Unschooled…"
"""0743294297""","""3282""",[],"""US""","""eng""","[{""7615"",""to-read""}, {""728"",""chick-lit""}, … {""6"",""bookworm-bitches""}]","""""","""false""","""3.49""","""B002ENBLOK""","[""6604176"", ""6054190"", … ""3134684""]","""Addie Downs an…","""Hardcover""","""https://www.go…","[{""9212"",""""}]","""Atria Books""","""368""","""14""","""9780743294294""","""7""","""""","""2009""","""https://www.go…","""https://s.gr-a…","""6066819""","""51184""","""6243154""","""Best Friends F…","""Best Friends F…"
"""0850308712""","""5""",[],"""US""","""""","[{""32"",""to-read""}, {""3"",""runes""}, … {""1"",""magick-and-myth""}]","""""","""false""","""3.40""","""""",[],"""""","""""","""https://www.go…","[{""149918"",""""}]","""""","""""","""""","""9780850308716""","""""","""""","""""","""https://www.go…","""https://images…","""287140""","""15""","""278577""","""Runic Astrolog…","""Runic Astrolog…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""0563553014""","""3""","[""618749""]","""US""","""eng""","[{""11"",""to-read""}, {""5"",""non-fiction""}, … {""1"",""audible""}]","""""","""false""","""4.05""","""""",[],"""The award-winn…","""Audio Cassette…","""https://www.go…","[{""4015"",""""}, {""14033"",""""}, … {""137454"",""Narrator""}]","""BBC Audiobooks…","""3""","""16""","""9780563553014""","""9""","""BBC Radio Coll…","""1999""","""https://www.go…","""https://images…","""3084038""","""12""","""3115103""","""This Sceptred …","""This Sceptred …"
"""178092870X""","""2""",[],"""US""","""eng""","[{""702"",""to-read""}, {""2"",""mystery""}, … {""1"",""would-love-to-win""}]","""""","""false""","""3.50""","""""","[""12064253"", ""25017213"", … ""23012660""]","""Sir Arthur Con…","""Paperback""","""https://www.go…","[{""2448"",""""}, {""3460250"",""""}]","""MX Publishing""","""148""","""26""","""9781780928708""","""8""","""2nd Edition""","""2015""","""https://www.go…","""https://images…","""26168430""","""6""","""46130263""","""Sherlock Holme…","""Sherlock Holme…"
"""0765197456""","""6""",[],"""US""","""""","[{""37"",""to-read""}, {""12"",""poetry""}, … {""1"",""my-collection""}]","""""","""false""","

In [4]:
books_df = books_df.select(['text_reviews_count', 'average_rating', 'num_pages', 'publication_year', 'book_id', 'ratings_count'])

In [5]:
books_df.collect()

text_reviews_count,average_rating,num_pages,publication_year,book_id,ratings_count
str,str,str,str,str,str
"""1""","""4.00""","""256""","""1984""","""5333265""","""3"""
"""6""","""3.23""","""""","""2001""","""1333909""","""10"""
"""7""","""4.03""","""600""","""1987""","""7327624""","""140"""
"""3282""","""3.49""","""368""","""2009""","""6066819""","""51184"""
"""5""","""3.40""","""""","""""","""287140""","""15"""
…,…,…,…,…,…
"""3""","""4.05""","""3""","""1999""","""3084038""","""12"""
"""2""","""3.50""","""148""","""2015""","""26168430""","""6"""
"""6""","""4.00""","""96""","""1996""","""2342551""","""36"""


In [6]:
books_df.dtypes

[String, String, String, String, String, String]

In [9]:
books_df = books_df.cast({'average_rating': pl.Float32, 'num_pages': pl.Int16, 'publication_year': pl.Int16, 'book_id': pl.Int64,
                          'ratings_count': pl.Int32, 'text_reviews_count': pl.Float64}, strict=False)

In [10]:
books_df.collect()

text_reviews_count,average_rating,num_pages,publication_year,book_id,ratings_count
f64,f32,i16,i16,i64,i32
1.0,4.0,256,1984,5333265,3
6.0,3.23,null,2001,1333909,10
7.0,4.03,600,1987,7327624,140
3282.0,3.49,368,2009,6066819,51184
5.0,3.4,null,null,287140,15
…,…,…,…,…,…
3.0,4.05,3,1999,3084038,12
2.0,3.5,148,2015,26168430,6
6.0,4.0,96,1996,2342551,36


In [11]:
books_df = books_df.with_columns([pl.col('num_pages').fill_null(strategy='mean').alias('num_pages'),
                                 pl.col('publication_year').fill_null(strategy='mean').alias('publication_year')])
books_df = books_df.drop_nulls()

In [12]:
books_df.collect()

text_reviews_count,average_rating,num_pages,publication_year,book_id,ratings_count
f64,f32,i16,i16,i64,i32
1.0,4.0,256,1984,5333265,3
6.0,3.23,262,2001,1333909,10
7.0,4.03,600,1987,7327624,140
3282.0,3.49,368,2009,6066819,51184
5.0,3.4,262,2007,287140,15
…,…,…,…,…,…
3.0,4.05,3,1999,3084038,12
2.0,3.5,148,2015,26168430,6
6.0,4.0,96,1996,2342551,36
